# Download mp3 files

The sections below allow us to provide a link to download mp3 files after specifying the singer.

The reason why we did not choose `requests.get()` to automatically download the audio files is because the access is often denied. **if can be fixed : )**

In [ ]:
# source:
# https://blackstarxing.github.io/2020-12-12-download-songs-by-python.html

from urllib import request
from bs4 import BeautifulSoup
import re
import requests
import time
import getpass
import os
import time

We are going to extract the link from a popular chinese music platform called `NetEase Cloud Music`.

In [ ]:
# user name
user_name = getpass.getuser()

# A class that help us extract url
class Music(object):
    def __init__(self, baseurl, path):
        # specify user agent
        head = {
            "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_2)"
            "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36"
        }
        # if you are using Chrome
        # you can find your user agent by typing
        # chrome://version/
        # into your browser
        baseurl = baseurl.replace("#/", "")
        self.baseurl = baseurl
        self.headers = head
        self.path = path

    def main(self):
        html = self.askurl()
        bs4 = self.analysis(html)
        id = self.matching(bs4)
        self.save(id)

    def askurl(self):
        req = request.Request(url=self.baseurl, headers=self.headers)
        response = request.urlopen(req)
        html = response.read().decode("utf-8")
        return html

    def analysis(self, html):
        soup = BeautifulSoup(html, "html.parser")
        bs4 = soup.find_all("li")
        bs4 = str(bs4)
        return bs4

    def matching(self, bs4):
        rule = re.compile(r'href="/song\?id=(\d*?)"', re.S)
        id = re.findall(rule, bs4)
        return id
    
    def extract(self, id_num):
        """id_num is a number,
            it has to be converted
            into string for usage"""
        url = "http://music.163.com/song?id=" + str(id_num)
        req = request.Request(url=url, headers=self.headers)
        response = request.urlopen(req)
        html = response.read().decode("utf-8")
        soup = BeautifulSoup(html, "html.parser")
        bs4 = soup.find_all("title")
        bs4 = str(bs4)
        rule = re.compile(
            r'<title>(.*?) - (.*?) - 单曲 - 网易云音乐</title>', re.S)
        name = re.findall(rule, bs4)
        name = name[0]
        songname = name[0].replace(r"/", "_")
        singername = name[1].replace(r"/", "_")
        print("Accessing：" + songname + " - " + singername + "……")
        saveurl = "http://music.163.com/song/media/outer/url?id=" + str(id_num)
        print("Please click on the following link: \n " + saveurl)
    
    def save(self, id):  
        if len(id) > 5:
            for i in range(5):
                self.extract(id[i])
        else:
            for i in range(len(id)):
                self.extract(id[i])
        return

In [ ]:
if __name__ == "__main__":
    try:
        artistid = getArtistId()
        artisturl = "http://music.163.com/#/artist?id=" + str(artistid)  # download url
    except:
        print('Wrong input! Try an integer from 1 to 5.')
    # path = "/Users/" + user_name + "/Downloads/music/"  # Save the path for downloading

    # # if the file under 'path' is not found
    # # then create the file
    # isExists=os.path.exists(path)
    
    # if not isExists:
    #     os.makedirs(path) 

    artist_demo = Music(artisturl, path)
    artist_demo.main()
    print("\n Successful extraction.")

Wrong input! Try an integer from 1 to 5.


NameError: name 'artisturl' is not defined

In [ ]:
try:
    content = input('Please specify singer\'s name: ')
    searchUrl = "http://music.163.com/api/search/get/web?csrf_token=hlpretag=&hlposttag=&s=" + \
    content + "&type=100&offset=0&total=true&limit=20"
    result = requests.get(url=searchUrl).json()
except:
    print('Please try again! (eg. Adele)')

In [ ]:
artistList = result['result']['artists'][:5]

TypeError: string indices must be integers

In [ ]:
# enumerate() 函数用于将一个可遍历的数据对象(如列表、元组或字符串)组合为一个索引序列，同时列出数据和数据下标
for i,artist in enumerate(artistList):
    number = i + 1
    name = artist['name']
    if 'transNames' in artist:
        name = name + '(' + artist['transNames'][0] + ')'
    print("[" + str(number) + "]: " + name)

In [ ]:
artistNo = int(input('Please enter the number you choose: '))
return artistList[artistNo-1]['id']

# Lyric


In [ ]:
!sudo apt-get update
!sudo apt-get install -y jq

Hit:1 http://security.debian.org/debian-security buster/updates InRelease
Hit:2 http://deb.debian.org/debian buster InRelease
Hit:3 http://deb.debian.org/debian buster-updates InRelease




jq is already the newest version (1.5+dfsg-2+b1).
0 upgraded, 0 newly installed, 0 to remove and 19 not upgraded.


The above code did not work for me, I use `brew` instead to install apt-get on a MacOS system.
If you want to know how to insall `Homebrew` on your computer, you can visit the link below:

    https://docs.brew.sh/Installation

or:

    https://www.digitalocean.com/community/tutorials/how-to-install-and-use-homebrew-on-macos

In [ ]:
artist_name = input("Enter the artist's name: ").replace(' ', '%20')

**Query Parameters:**

1. query
    * artist
    * primarytype: Single
    * country: US
2. fmt = json
3. limit = 10

From the MusicBrainz database, retrieve 10 entries that satisfy the given parameters(artist, primarytype, country) in json format.

In [ ]:
!curl --request GET -o tracks_raw.json "https://musicbrainz.org/ws/2/recording/?query=artist:'"'$artist_name'"'%20AND%20primarytype:Single%20AND%20country:US&fmt=json&limit=10"
!jq < tracks_raw.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 25952  100 25952    0     0  22665      0  0:00:01  0:00:01 --:--:-- 22665
zsh:1: command not found: jq


In [ ]:
!jq '.recordings' < tracks_raw.json >tracks_rec.json
!jq '.[]|.title'< tracks_rec.json > tracks.json

In [ ]:
tracks_file = open("tracks.json")
tracks = []
for line in tracks_file:
    tracks.append(line.replace('"','').replace('\n',''))
print(*tracks, sep = "\n")
song = tracks[int(input("Enter the song (1-10) that you would like to search the lyrics for: "))-1].replace(' ','%20')

In [ ]:
!curl --request GET "https://api.lyrics.ovh/v1/$artist_name/$song"

In [ ]:
!curl --request GET -o lyrics_raw.json "https://api.lyrics.ovh/v1/$artist_name/$song"
!jq '.lyrics' < lyrics_raw.json > lyrics.json
lyrics = open("lyrics.json").read()

In [ ]:
print(*lyrics.replace('\\r','').replace('\\\'','\'').split('\\n'), sep = "\n")

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=34182c1e-ced2-433c-9676-54ec654bbd88' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>